# Présentation de la boite à outil Julia pour l'Algèbre Max-Plus

L'algebre Max-Plus redéfinit les opérateurs addition et multiplication par les opérateurs max et addition.

$$a \oplus b = \max(a,b)$$
$$a \otimes b = a + b$$

L'algebre Max-Plus permet de modéliser et d'évaluer les performances de systèmes à évènements discrets (réseaux de transport ou de télécom, systèmes de production), mais également dans des domaines tels que et théorie de la décision, recherche opérationnelle.

Une boite à outils Max-Plus est directement intégrée dans le logiciel [ScicosLab](http://www.scicoslab.org) qui est le fork officiel de Scilab maintenu par les anciens développeurs. Dans ce document nous allons utiliser le langage Julia (version >= 1.0.3 bien qu'une version > 1.3 soit plus recommandée car elle apporte des correctifs sur les matrices creuses) ainsi que ce [package MaxPlus](https://github.com/Lecrapouille/MaxPlus.jl). Ce document permet à la fois d'introduire cette algèbre et de présenter les fonctions de base pour Julia.

Pour commencer, nous allons télécharger cette boite à outils Julia. Vous avez deux options:
- si vous travaillez depuis une session REPL Julia, vous pouvez télécharger la dernière version sur GitHub :

In [ ]:
# using Pkg; Pkg.add(PackageSpec(url="https://github.com/Lecrapouille/MaxPlus.jl"));
# using MaxPlus;

- Soit depuis le gestionaire de package Julia:

In [ ]:
# using Pkg; Pkg.add("MaxPlus")

- Soit depuis ce document Jupyter (le code précédent ne fonctionnant pas avec un document Jupyter) :

In [1]:
push!(LOAD_PATH, pwd())
using MaxPlus

Pour le moment et dans un soucis pédagogique on active un mode d'affichage particulier des nombres max-plus. L'explication sera donnée plus tard.

In [3]:
mp_change_display(0);

Verifions le version de votre Julia. Une version > 1.3 est le plus recommandée car elle apporte des correctifs sur les matrices creuses. Certains correctifs sont pris en compte avec le package MaxPlus.

In [11]:
versioninfo()

Julia Version 1.0.3
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: AMD Ryzen 7 1800X Eight-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, znver1)


---
## Les types Max-Plus

Crééons quelques nombres max-plus sous Julia. Ces nombres sont templatés (Float ou Int). Par exemple on écrira :

In [4]:
MP{Float64}(42)

42.0

Ou bien plus court :

In [5]:
a = MP(1.0)
b = MP(3.0)
typeof(a)

MP{Float64}

Même idée pour les entiers :

In [6]:
c = MP(3)
typeof(c)

MP{Int64}

Les nombres max-plus contaminent sur les autres nombres : ils convertisent un nombre non max-plus en nombre max-plus via les opérateurs :

In [7]:
d = 1.0
[typeof(c) typeof(d) typeof(c + d)]

1×3 Array{DataType,2}:
 MP{Int64}  Float64  MP{Float64}

Nous voyons que l'addition max-plus à transformer la variable c MP{Int64} en MP{Float64}.

Même idée pour les matrices denses (et creuses) :

In [8]:
[MP(1.0) 2; 3 4]

2×2 Array{MP{Float64},2}:
 1.0  2.0
 3.0  4.0

MP(1.0) de type MP{Float64} a contaminé les nombres entiers *classiques* 2 3 et 4 en MP{Float64}.

Voici une façon de le faire plus simple :

In [9]:
MP([1.0 2; 3 4])

2×2 Array{MP{Float64},2}:
 1.0  2.0
 3.0  4.0

Voici un autre exmple plus complexe de matrice contenant des nombres infinis $-\infty$ (qui sont du type MP{Float}):

In [10]:
I = MP([-Inf 0; 0 -Inf])

2×2 Array{MP{Float64},2}:
 -Inf       0.0
    0.0  -Inf  

Si l'on désire convertir un nombre max-plus en un nombre ordinaire (au sens algèbre classique, celle que l'on utilise tous les jours) :

In [12]:
e = plustimes(a)
[typeof(a) typeof(e) typeof(MP(e))]

1×3 Array{DataType,2}:
 MP{Float64}  Float64  MP{Float64}

Nous voyons que la variable e étant du type MP{Float64} est devenue de type Float64. Elle se comportera désormais comme un vrai nombre :

In [13]:
[e       a
 e + e   a + a]

2×2 Array{MP{Float64},2}:
 1.0  1.0
 2.0  1.0

On peut vouloir convertir les valeurs pour l'algèbre min-plus (les signes des $-\infty$ sont inversés) :

In [79]:
minplus(MP(-Inf))

Inf

---
### Constantes Julia max-plus

Certaines constantes max-plus sont prédéfinies:
- mp0 pour $-\infty$ (que l'on note en général $\varepsilon$),
- mp1 pour 0 (que l'on note en général $e$),
- mptop pour $+\infty$ (utilisé pour l'algèbre min-plus).

In [14]:
[mp0 mp1 mptop; mpzero(Float64) mpone(Float64) mptop]

2×3 Array{MP{Float64},2}:
 -Inf  0.0  Inf
 -Inf  0.0  Inf

Le template peuvent être changé :

In [18]:
[mpzero(Int64) mpone(Int64)]

1×2 Array{MP{Int64},2}:
 -9223372036854775808  0

Mais attention mp0, mp1 et mptop sont prédéfinis avec le type MP{Float}.

---
### Affichage des nombres max-plus

Il y a 4 styles d'affichage possibles:
- Style 0: les nombres $-\infty$ et les zéros sont affichés tels quels.
- Style 1 ou 2: les nombres $-\infty$ sont affiché sous la forme d'un point.
- Style 3 ou 4: les nombres $-\infty$ sont affichés sous la forme $\varepsilon$.
- Style 1 ou 3: les zeros sont affichés $0$.
- Style 2 ou 4: les zeros sont affichés $e$.

Le style 2 est activé par défaut et correspond à l'affichage de ScicosLab. Notons que les nombres réels qui peuvent être écrits comme des entiers seront affichés comme des entiers. Ce choix en combinaison des diffèrents modes permet de comprésser l'affichage des matrices. En effet, il est commun en max-plus de devoir ma,ipuler et afficher de grosses matrices creuses.

In [24]:
mp_change_display(0)
I = MP([-Inf 0; 0 -Inf])

2×2 Array{MP{Float64},2}:
 -Inf       0.0
    0.0  -Inf  

In [25]:
mp_change_display(1)
I

2×2 Array{MP{Float64},2}:
  .  0 
 0    .

In [21]:
mp_change_display(2)
I

2×2 Array{MP{Float64},2}:
 .  e
 e  .

In [22]:
mp_change_display(3)
I

2×2 Array{MP{Float64},2}:
 ε  0
 0  ε

In [29]:
mp_change_display(4)
I

2×2 Array{MP{Float64},2}:
 ε    e
  e  ε 

On peut générer du code LaTeX à partir d'une matrice max-plus. la fonction mp_change_display() modifie également le code LaTeX généré.

In [30]:
LaTeX(stdout, I)

\left[
\begin{array}{*{20}c}
\varepsilon & e \\
e & \varepsilon \\
\end{array}
\right]


Qui donnera :

$$\left[
\begin{array}{*{20}c}
\varepsilon & e \\
e & \varepsilon \\
\end{array}
\right]$$

In [60]:
mp_change_display(0)
LaTeX(stdout, I)

\left[
\begin{array}{*{20}c}
-\infty & 0 \\
0 & -\infty \\
\end{array}
\right]


Donnera :
$$\left[
\begin{array}{*{20}c}
-\infty & 0 \\
0 & -\infty \\
\end{array}
\right]$$

---
### Max-Plus opérateur $\oplus$

L'opérateur addition est redéfini par l'opérateur max() qui est associatif, commutatif, a un élément neutre (noté $\varepsilon$) et est idempotent.

$$a \oplus b \triangleq \max(a,b)$$

In [34]:
[a b]

1×2 Array{MP{Float64},2}:
 1  3

In [33]:
a + b    # ≜ max(a, b) == max(1, 3) == 3

3

#### Commutativité

$$a \oplus b = b \oplus a$$
$$\triangleq$$
$$\max(a,b) = \max(b,a)$$

In [35]:
a + b == b + a

true

#### Associativité

$$a \oplus b \oplus c = (a \oplus b) \oplus c = a \oplus (b \oplus c)$$

In [36]:
a + b + c == (a + b) + c == a + (b + c)

true

#### Elément neutre $\varepsilon$

$$a \oplus \varepsilon = \varepsilon \oplus a = a$$
$$\triangleq$$
$$\max(a,-\infty) = \max(-\infty,a) = a$$

In [45]:
a + mp0 == mp0 + a == a

true

Notons que $0$ est neutre pour les nombres positifs :

In [46]:
a + 0 == 0 + a == a

true

#### Elément absorbant $\infty$

$$a \oplus \infty = \infty \oplus a$$
$$\triangleq$$
$$\max(a,\infty) = \max(\infty,a) = \infty$$

In [38]:
a + mptop == mptop + a == mptop

true

#### Idempotent

In [39]:
a + a    # ≜ max(a, a) == max(1, 1) == 1

1

---
### Max-Plus opérateur $\otimes$

L'opérateur multiplication est redéfini par l'opérateur addition qui est associatif, commutatif, a l'élément neutre $e$, l'élément absorbant $\varepsilon$ et est distributif sur $\oplus$.

In [41]:
a * b    # ≜ a + b == 1 + 3 == 4

4

#### Commutativité

$$a \otimes b = b \otimes a$$
$$\triangleq$$
$$a + b = b + a$$

In [42]:
a * b == b * a

true

#### Associativité

$$a \otimes b \otimes c = (a \otimes b) \otimes c = a \otimes (b \otimes c)$$

In [43]:
a * b * c == (a * b) * c == a * (b * c)

true

#### Elément neutre $e$

$$a \otimes e = e \otimes a = a$$
$$\triangleq$$
$$a + 0 = 0 + a = a$$

In [ ]:
a * mp1 == mp1 * a == a 

#### Elément absorbant $\varepsilon$

$$a \otimes \varepsilon = \varepsilon \otimes a = \varepsilon$$
$$\triangleq$$
$$a -\infty = -\infty + a = -\infty$$

In [ ]:
a * mp0 == mp0 * a == mp0

Par convention:

$$\infty \otimes \varepsilon = \varepsilon \otimes \infty = \varepsilon$$

In [50]:
# mptop * mp0 TODO shall return mp0

**FIXME**

#### N'est pas idempotent

In [ ]:
a * a    # ≜ a + a == 1 + 1 == 2

---
### Distributivité de $\otimes$ sur $\oplus$

$$a \otimes (b \oplus c) = (a \otimes b) \oplus (a \otimes c)$$
$$(b \oplus c) \otimes a = (b \otimes a) \oplus (c \otimes a)$$

$$a \oplus b \otimes c = a + \max(a, b)$$
$$a \otimes c \oplus b \otimes c = \max(a+c,b+c)$$

In [51]:
(a + b) * c == (a * c) + (b * c)     # => max(a, b) + c == max(a + c, b + c) 

true

In [52]:
(a * c) + (b * c)

6

---
### Produit matriciel

Les matrices peuvent être de type max-plus. Le produit matriciel correspond au produit matriciel avec les operateurs $+$ et $\times$ surchargés.

$$A=\begin{bmatrix}
4 & 3 \\
7 & -\infty
\end{bmatrix}\;,$$

$$A \otimes A = \begin{bmatrix}
4 \otimes 4 \oplus 7 \otimes7 & 4 \otimes 3 \oplus 3 \otimes -\infty \\
7 \otimes 4 \oplus -\infty \otimes 7 & 7 \otimes 3 \oplus -\infty \otimes -\infty
\end{bmatrix}\; = \begin{bmatrix}
10 & 7 \\
11 & 10
\end{bmatrix}\; = A^2.$$

In [54]:
A = MP([4 3; 7 -Inf])
A * A

2×2 Array{MP{Float64},2}:
 10   7
 11  10

In [55]:
A * A == A^2

true

En algébre max-plus l'opérateur puissance devient une simple multiplication :

In [64]:
MP(2.0)^5   # ==> 2.0 * 5.0 

10.0

In [61]:
MP(2.0)^0

0.0

S'applique également aux matrices :

In [62]:
A^5

2×2 Array{MP{Float64},2}:
 24.0  23.0
 27.0  24.0

In [63]:
A^0

2×2 Array{MP{Float64},2}:
    0.0  -Inf  
 -Inf       0.0

---
### Quelques matrice utiles

Matrice identité de dimension 2x2 :

$$\left[
\begin{array}{*{20}c}
e & \varepsilon \\
\varepsilon & e \\
\end{array}
\right]$$

In [73]:
I = mpeye(Float64, 2,2)

2×2 Array{MP{Float64},2}:
    0.0  -Inf  
 -Inf       0.0

Matrices remplies uniquement de $e$ :

$$\left[
\begin{array}{*{20}c}
e & e \\
e & e \\
\end{array}
\right]$$

In [74]:
O = mpones(Float64, 2,2)

2×2 Array{MP{Float64},2}:
 0.0  0.0
 0.0  0.0

Matrices remplies de $\varepsilon$ :

$$\left[
\begin{array}{*{20}c}
\varepsilon & \varepsilon \\
\varepsilon & \varepsilon \\
\end{array}
\right]$$

In [77]:
Z = full(mpzeros(Float64, 2,2))

2×2 Array{MP{Float64},2}:
 -Inf  -Inf
 -Inf  -Inf

**Note :** On notera la présence de la fonction full(). En effet la fonction mpzeros() retournera une matrice creuse (que l'on verra plus loin dans ce document). 

---
### Matrices creuses

Une matrice creuse est une matrice contenant beaucoup de zéros. La structure des matrices creuses évitent de garder en mémoire ces zéros. La structure internet d'une matrices creuse en Julia est trois vecteurs : un vecteur pour garder les données non nulles et deux vecteurs pour mémoriser les indices de ces données.

Beaucoup d'exemples dans la natures sont représentés par une matrice creuse. Par exemple pour stocker un graphe on peut utiliser des matrices (dites d'ajacence). Le réseau routier est un graphe: les routes sont les arcs et les carrefours sont les noeuds. En général un carrefour permet de rejoindre 3 ou 4 routes. On utilisera une matrice d'ajacence creuse.

En algébre classique les zéros sont 0 (pour les entiers) ou 0.0 (réels) mais en max-plus les zéros valent $-\infty$.

Pour la suite de ce document on aura besoin d'un package Julia de base supplémentaire :

In [93]:
using SparseArrays

**Note: On préfèrera la version Julia > 1.3 qui corrige des bugs dans les matrices creuses. Les matrices creuses max-plus ne stockent pas les valeurs $-\infty$ mais dans les versions précédentes de Julia les valeurs zéros étaient écrites sous la forme litérale $0$ au lieu de la fonction zero() entrainant des cas d'erreurs. Le package MaxPlus en corrige certains mais il n'est pas garantit qu'il les corrige tous !**

Création d'une matrice creuse de taille 2x2 max-plus vide :

In [94]:
Z = mpzeros(Float64, 2,2)

2×2 SparseMatrixCSC{MP{Float64},Int64} with 0 stored entries

Création d'une matrice creuse avec des données non max-plus:

In [98]:
MP(sparse([1, 2, 3], [1, 2, 3], [5, 2, 6]))

3×3 SparseMatrixCSC{MP{Int64},Int64} with 3 stored entries:
  [1, 1]  =  5
  [2, 2]  =  2
  [3, 3]  =  6

Création d'une matrice creuse avec des données max-plus :

In [97]:
sparse([1, 2, 3], [1, 2, 3], MP([5, 2, 6]))

3×3 SparseMatrixCSC{MP{Int64},Int64} with 3 stored entries:
  [1, 1]  =  5
  [2, 2]  =  2
  [3, 3]  =  6

Julia autorise de créer des matrices contenant des éléments zéros :

In [99]:
A = MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]))

3×3 SparseMatrixCSC{MP{Float64},Int64} with 3 stored entries:
  [1, 1]  =  -Inf
  [2, 2]  =  2.0
  [3, 3]  =  0.0

Ici le -Inf est bien stocké. Mais il n'est pas utilsé. Par exemple :

In [103]:
B = MP(sparse([2, 3], [2, 3], [2.0, 0]))

3×3 SparseMatrixCSC{MP{Float64},Int64} with 2 stored entries:
  [2, 2]  =  2.0
  [3, 3]  =  0.0

In [104]:
A == B

true

Le $-\infty$ n'est pas testé.

Conversion d'une matrice creuse en matrice pleine :

In [84]:
full(Z)  # ou bien dense(Z) ou bien Array(Z)

2×2 Array{MP{Float64},2}:
 -Inf  -Inf
 -Inf  -Inf

On remarquera que cette matrice contient que des $-\infty$. En effet, ils correspondent aux 0 éliminés des matrices creuses en algébre classique. Une matrice creuse max-plus ne stocke pas les nombres max-plus $-\infty$ (**note:** enfin jusqu'à Julia > 1.3 car les versions précédentes avaient un bug elles confondaient 0 et zero(T) avec T template de type MP). 

Conversion d'une matrice creuse max-plus à partir d'une matrice pleine en algébre classique :

In [105]:
A = [4 0; 7 -Inf]

2×2 Array{Float64,2}:
 4.0     0.0
 7.0  -Inf  

In [116]:
mpsparse(A)

2×2 SparseMatrixCSC{MP{Float64},Int64} with 3 stored entries:
  [1, 1]  =  4.0
  [2, 1]  =  7.0
  [1, 2]  =  0.0

On remarquera que le $-\infty$ a disparu. Si on voulait le garder : **FIXME**

In [109]:
# mpsparse(A; preserve=true) TODO

Que l'on peut éléminer par :

In [121]:
MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]), preserve=false)

3×3 SparseMatrixCSC{MP{Float64},Int64} with 1 stored entry:
  [2, 2]  =  2.0

In [119]:
MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]), preserve=true)

3×3 SparseMatrixCSC{MP{Float64},Int64} with 3 stored entries:
  [1, 1]  =  -Inf
  [2, 2]  =  2.0
  [3, 3]  =  0.0

---
### Conversion de matrices max-plus

On peut vouloir convertir les valeurs pour l'algèbre min-plus (les signes des $-\infty$ sont inversés) :

In [80]:
minplus(Z)

2×2 Array{MP{Float64},2}:
 Inf  Inf
 Inf  Inf

On peut vouloir convertir une matrice max-plus en valeurs non max-plus :

In [81]:
plustimes(Z)

2×2 Array{Float64,2}:
 -Inf  -Inf
 -Inf  -Inf

---
### Trace d'une matrice

Est la somme max-plus des élements diagnonaux.

In [65]:
mptrace(A)

4.0

In [66]:
mptrace(A) == A[1,1] + A[2,2]

true

---
### Norme d'une matrice

In [72]:
mpnorm(mpeye(Float64, 2,2))

Inf

---
### Algorithme $A^*$ (A star)

---
### Valeurs propres d'une matrice